# Terra Signal Hackathon
This notebook is provided as a starting point. Feel free to use it, discard it, modify it, or pretend it doesn't exist.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file using pandas
file_path = "./history.csv"
df = pd.read_csv(file_path)
df.head().transpose()

In [0]:
df.info()

Limpeza dos dados

In [0]:
pdf["tokens"] = pdf["CustomerFeedback_clean"].str.split()

# 0 = não churn (positivo), 1 = churn (negativo)
pdf_pos = pdf[pdf["Churn"] == 0]
pdf_neg = pdf[pdf["Churn"] == 1]

In [0]:
from collections import Counter

pos_counts = Counter(
    w
    for tokens in pdf_pos["tokens"]
    for w in tokens
)

neg_counts = Counter(
    w
    for tokens in pdf_neg["tokens"]
    for w in tokens
)

len(pos_counts), len(neg_counts)

In [0]:
import numpy as np
import pandas as pd

all_words = set(pos_counts) | set(neg_counts)

total_pos = sum(pos_counts.values())
total_neg = sum(neg_counts.values())
V = len(all_words)  # vocabulário para suavização

rows = []
for w in all_words:
    pos = pos_counts.get(w, 0)
    neg = neg_counts.get(w, 0)
    total = pos + neg

    # filtra palavras muito raras (ajuste o limiar)
    if total < 5:
        continue

    # probabilidades suavizadas
    p_pos = (pos + 1) / (total_pos + V)
    p_neg = (neg + 1) / (total_neg + V)

    log_odds = np.log(p_neg / p_pos)

    rows.append((w, pos, neg, total, log_odds))

word_stats = pd.DataFrame(
    rows,
    columns=["word", "pos_count", "neg_count", "total", "log_odds"]
)

word_stats.sort_values("log_odds", ascending=False).head(10)

In [0]:
%pip install wordcloud

In [0]:
# principais negativas
neg_words = (
    word_stats.sort_values("log_odds", ascending=False)
              .head(100)
)

# principais positivas
pos_words = (
    word_stats.sort_values("log_odds", ascending=True)
              .head(100)
)

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# dicionários palavra -> peso (uso total ou |log_odds|)
pos_freq = {row.word: row.total for _, row in pos_words.iterrows()}
neg_freq = {row.word: row.total for _, row in neg_words.iterrows()}

In [0]:
wc_pos = WordCloud(
    width=800,
    height=400,
    background_color="white"
).generate_from_frequencies(pos_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a NÃO churn (positivas)")
display(plt.gcf())
plt.close()

In [0]:
wc_neg = WordCloud(
    width=800,
    height=400,
    background_color="white"
).generate_from_frequencies(neg_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a churn (negativas)")
display(plt.gcf())
plt.close()

In [0]:
spark_word_stats = spark.createDataFrame(word_stats)
spark_word_stats.write.mode("overwrite").saveAsTable("workspace.churn.word_polarity")

In [0]:
%pip install scikit-learn wordcloud

In [0]:
# ============================================
# Naive Bayes de palavras (positivo x negativo) + WordCloud
# Baseado na ideia do projeto Spark, mas em Python puro (sem Spark)
# Usa a coluna "CustomerFeedback_clean" e "Churn" de um CSV
#  - Churn = 0  -> feedback positivo (não churn)
#  - Churn = 1  -> feedback negativo (churn)
# ============================================

# Instale as dependências (rode UMA vez no ambiente):
# pip install pandas numpy wordcloud matplotlib

import pandas as pd
import numpy as np
from collections import Counter
from math import log10
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# ------------------------------------------------
# 1. Carregar dados
# ------------------------------------------------
# Ajuste o caminho do arquivo aqui:
CSV_PATH = "history_clean.csv"

df = pd.read_csv(CSV_PATH)

# Garante que as colunas necessárias existem
col_label = "Churn"
col_text  = "CustomerFeedback_clean"

if col_label not in df.columns or col_text not in df.columns:
    raise ValueError(f"O CSV precisa ter as colunas '{col_label}' e '{col_text}'.")

# Remove linhas sem texto
df = df.dropna(subset=[col_text])

# Garante tipos corretos
df[col_label] = df[col_label].astype(int)
df[col_text]  = df[col_text].astype(str)

# ------------------------------------------------
# 2. Construir contagens de palavras por classe (0 = não churn, 1 = churn)
# ------------------------------------------------
pos_counter = Counter()  # não churn (positivo)
neg_counter = Counter()  # churn (negativo)

for _, row in df.iterrows():
    label = row[col_label]
    text  = row[col_text].lower()
    tokens = text.split()
    if label == 0:
        for w in tokens:
            pos_counter[w] += 1
    elif label == 1:
        for w in tokens:
            neg_counter[w] += 1
    # Se tiver outros valores de Churn, são ignorados

# ------------------------------------------------
# 3. Calcular log P(palavra | classe) com Laplace
# ------------------------------------------------
vocab = set(pos_counter.keys()) | set(neg_counter.keys())
V = len(vocab)

total_pos = sum(pos_counter.values())
total_neg = sum(neg_counter.values())

alpha = 1  # Laplace smoothing

# modelo: word -> (log_prob_pos, log_prob_neg)
modelo = {}

for w in vocab:
    c_pos = pos_counter.get(w, 0)
    c_neg = neg_counter.get(w, 0)

    # P(w | classe) com Laplace
    p_pos = (c_pos + alpha) / (total_pos + alpha * V)
    p_neg = (c_neg + alpha) / (total_neg + alpha * V)

    log_p_pos = log10(p_pos)
    log_p_neg = log10(p_neg)

    modelo[w] = (log_p_pos, log_p_neg)

# ------------------------------------------------
# 4. Função classificador (opcional, caso queira classificar textos)
# ------------------------------------------------
def classificar_texto(texto):
    """
    Retorna 0 (não churn / positivo) ou 1 (churn / negativo)
    com base no modelo Naive Bayes de palavras.
    """
    texto = str(texto).lower()
    tokens = texto.split()

    score_pos = 0.0
    score_neg = 0.0

    for p in tokens:
        if p in modelo:
            log_p_pos, log_p_neg = modelo[p]
            score_pos += log_p_pos
            score_neg += log_p_neg

    # Se a soma de log-probs for maior em churn, retorna 1 (negativo)
    return 1 if score_neg > score_pos else 0

# Exemplo de uso:
# print(classificar_texto("service is terrible and very slow"))
# print(classificar_texto("great support, very happy"))

# ------------------------------------------------
# 5. Medir polaridade de cada palavra (positivo x negativo)
#    score = log P(w | churn=1) - log P(w | churn=0)
#    score > 0 -> palavra mais associada a churn (negativa)
#    score < 0 -> palavra mais associada a não churn (positiva)
# ------------------------------------------------
palavras = []
log_pos_list = []
log_neg_list = []
score_list   = []
count_list   = []

for w, (log_p_pos, log_p_neg) in modelo.items():
    score = log_p_neg - log_p_pos  # alinhado com "negativo" = churn
    total_count = pos_counter.get(w, 0) + neg_counter.get(w, 0)
    palavras.append(w)
    log_pos_list.append(log_p_pos)
    log_neg_list.append(log_p_neg)
    score_list.append(score)
    count_list.append(total_count)

word_stats = pd.DataFrame({
    "word": palavras,
    "log_prob_pos": log_pos_list,
    "log_prob_neg": log_neg_list,
    "score": score_list,
    "count": count_list
})

# Opcional: filtrar palavras muito raras
min_count = 5
word_stats = word_stats[word_stats["count"] >= min_count].reset_index(drop=True)

# ------------------------------------------------
# 6. Selecionar top palavras positivas e negativas
# ------------------------------------------------
TOP_N = 100

# Negativas: mais associadas a churn (score alto)
neg_words = word_stats.sort_values("score", ascending=False).head(TOP_N)

# Positivas: mais associadas a não churn (score bem negativo)
pos_words = word_stats.sort_values("score", ascending=True).head(TOP_N)

print("Top 10 palavras NEGATIVAS (churn):")
print(neg_words[["word", "score", "count"]].head(10))
print("\nTop 10 palavras POSITIVAS (não churn):")
print(pos_words[["word", "score", "count"]].head(10))

# ------------------------------------------------
# 7. WordCloud das principais palavras negativas e positivas
#    Usamos |score| como peso (quanto mais forte a polaridade,
#    maior a palavra na nuvem)
# ------------------------------------------------
neg_freq = {row.word: abs(row.score) for _, row in neg_words.iterrows()}
pos_freq = {row.word: abs(row.score) for _, row in pos_words.iterrows()}

# Nuvem de palavras NEGATIVAS (churn) – em VERMELHO
wc_neg = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Reds"  # <- paleta vermelha
).generate_from_frequencies(neg_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a churn (negativas)")
plt.show()

# Nuvem de palavras POSITIVAS (não churn) – em VERDE
wc_pos = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Greens"  # <- paleta verde
).generate_from_frequencies(pos_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a NÃO churn (positivas)")
plt.show()

# ------------------------------------------------
# 8. (Opcional) Salvar a tabela de polaridade em CSV
# ------------------------------------------------
# word_stats.to_csv("word_polarity_naive_bayes.csv", index=False)

In [0]:
# ------------------------------------------------
# 4.1. Avaliar acurácia do modelo Naive Bayes
# ------------------------------------------------

# Cria uma coluna com a previsão do modelo para cada feedback
df["pred_nb"] = df[col_text].apply(classificar_texto)

# Acurácia global
accuracy = (df["pred_nb"] == df[col_label]).mean()
print(f"Acurácia global do Naive Bayes: {accuracy:.4f}")

# Matriz de confusão
confusion = pd.crosstab(df[col_label], df["pred_nb"],
                        rownames=["Churn real"],
                        colnames=["Churn previsto"])
print("\nMatriz de confusão:")
print(confusion)

# Acurácia separada por classe (opcional)
acc_por_classe = (
    df.assign(acerto=(df["pred_nb"] == df[col_label]).astype(int))
      .groupby(col_label)["acerto"]
      .mean()
)
print("\nAcurácia por classe (0 = não churn, 1 = churn):")
print(acc_por_classe)

In [0]:
# ============================================
# Naive Bayes de palavras (positivo x negativo) + WordCloud
# Agora com split TREINO / TESTE e avaliação de acurácia em TESTE
#
# Usa a coluna "CustomerFeedback_clean" e "Churn" de um CSV
#  - Churn = 0  -> feedback positivo (não churn)
#  - Churn = 1  -> feedback negativo (churn)
# ============================================

# Dependências (rode UMA vez):
# pip install pandas numpy wordcloud matplotlib scikit-learn

import pandas as pd
import numpy as np
from collections import Counter
from math import log10
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ------------------------------------------------
# 1. Carregar dados
# ------------------------------------------------
CSV_PATH = "history_clean.csv"

df = pd.read_csv(CSV_PATH)

col_label = "Churn"
col_text  = "CustomerFeedback_clean"

if col_label not in df.columns or col_text not in df.columns:
    raise ValueError(f"O CSV precisa ter as colunas '{col_label}' e '{col_text}'.")

df = df.dropna(subset=[col_text])

df[col_label] = df[col_label].astype(int)
df[col_text]  = df[col_text].astype(str)

# ------------------------------------------------
# 1.1 Split TREINO / TESTE
# ------------------------------------------------
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df[col_label]  # mantém proporção de classes
)

print(f"Tamanho treino: {len(df_train)} | Tamanho teste: {len(df_test)}")

# ------------------------------------------------
# 2. Construir contagens de palavras por classe (apenas TREINO)
# ------------------------------------------------
pos_counter = Counter()  # não churn (positivo)
neg_counter = Counter()  # churn (negativo)

for _, row in df_train.iterrows():
    label = row[col_label]
    text  = row[col_text].lower()
    tokens = text.split()
    if label == 0:
        for w in tokens:
            pos_counter[w] += 1
    elif label == 1:
        for w in tokens:
            neg_counter[w] += 1

# ------------------------------------------------
# 3. Calcular log P(palavra | classe) com Laplace (modelo treinado)
# ------------------------------------------------
vocab = set(pos_counter.keys()) | set(neg_counter.keys())
V = len(vocab)

total_pos = sum(pos_counter.values())
total_neg = sum(neg_counter.values())

alpha = 1  # Laplace smoothing

# modelo: word -> (log_prob_pos, log_prob_neg)
modelo = {}

for w in vocab:
    c_pos = pos_counter.get(w, 0)
    c_neg = neg_counter.get(w, 0)

    p_pos = (c_pos + alpha) / (total_pos + alpha * V)
    p_neg = (c_neg + alpha) / (total_neg + alpha * V)

    log_p_pos = log10(p_pos)
    log_p_neg = log10(p_neg)

    modelo[w] = (log_p_pos, log_p_neg)

# ------------------------------------------------
# 4. Função classificador
# ------------------------------------------------
def classificar_texto(texto):
    """
    Retorna 0 (não churn / positivo) ou 1 (churn / negativo)
    com base no modelo Naive Bayes de palavras.
    """
    texto = str(texto).lower()
    tokens = texto.split()

    score_pos = 0.0
    score_neg = 0.0

    for p in tokens:
        if p in modelo:
            log_p_pos, log_p_neg = modelo[p]
            score_pos += log_p_pos
            score_neg += log_p_neg

    return 1 if score_neg > score_pos else 0

# ------------------------------------------------
# 4.1 Avaliar acurácia em TESTE
# ------------------------------------------------
df_test = df_test.copy()
df_test["pred_nb"] = df_test[col_text].apply(classificar_texto)

accuracy_test = (df_test["pred_nb"] == df_test[col_label]).mean()
print(f"\nAcurácia em TESTE do Naive Bayes: {accuracy_test:.4f}")

confusion_test = pd.crosstab(
    df_test[col_label],
    df_test["pred_nb"],
    rownames=["Churn real"],
    colnames=["Churn previsto"]
)
print("\nMatriz de confusão (TESTE):")
print(confusion_test)

acc_por_classe = (
    df_test.assign(acerto=(df_test["pred_nb"] == df_test[col_label]).astype(int))
           .groupby(col_label)["acerto"]
           .mean()
)
print("\nAcurácia por classe (TESTE) (0 = não churn, 1 = churn):")
print(acc_por_classe)

# ------------------------------------------------
# 5. Medir polaridade de cada palavra (usando o modelo treinado)
# ------------------------------------------------
palavras = []
log_pos_list = []
log_neg_list = []
score_list   = []
count_list   = []

for w, (log_p_pos, log_p_neg) in modelo.items():
    score = log_p_neg - log_p_pos  # >0: mais associada a churn
    total_count = pos_counter.get(w, 0) + neg_counter.get(w, 0)
    palavras.append(w)
    log_pos_list.append(log_p_pos)
    log_neg_list.append(log_p_neg)
    score_list.append(score)
    count_list.append(total_count)

word_stats = pd.DataFrame({
    "word": palavras,
    "log_prob_pos": log_pos_list,
    "log_prob_neg": log_neg_list,
    "score": score_list,
    "count": count_list
})

# Opcional: filtrar palavras muito raras
min_count = 5
word_stats = word_stats[word_stats["count"] >= min_count].reset_index(drop=True)

# ------------------------------------------------
# 6. Selecionar top palavras positivas e negativas
# ------------------------------------------------
TOP_N = 100

neg_words = word_stats.sort_values("score", ascending=False).head(TOP_N)
pos_words = word_stats.sort_values("score", ascending=True).head(TOP_N)

print("\nTop 10 palavras NEGATIVAS (churn):")
print(neg_words[["word", "score", "count"]].head(10))
print("\nTop 10 palavras POSITIVAS (não churn):")
print(pos_words[["word", "score", "count"]].head(10))

# ------------------------------------------------
# 7. WordCloud das principais palavras negativas e positivas
# ------------------------------------------------
neg_freq = {row.word: abs(row.score) for _, row in neg_words.iterrows()}
pos_freq = {row.word: abs(row.score) for _, row in pos_words.iterrows()}

wc_neg = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Reds"
).generate_from_frequencies(neg_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a churn (negativas)")
plt.show()

wc_pos = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Greens"
).generate_from_frequencies(pos_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a NÃO churn (positivas)")
plt.show()

# ------------------------------------------------
# 8. (Opcional) Salvar a tabela de polaridade em CSV
# ------------------------------------------------
# word_stats.to_csv("word_polarity_naive_bayes.csv", index=False)

In [0]:
# ------------------------------------------------
# Avaliar acurácia em TESTE
# ------------------------------------------------
df_test = df_test.copy()
df_test["pred_nb"] = df_test[col_text].apply(classificar_texto)

# Acurácia global
accuracy_test = (df_test["pred_nb"] == df_test[col_label]).mean()
print(f"Acurácia em TESTE do Naive Bayes: {accuracy_test:.4f}")

# Matriz de confusão
confusion_test = pd.crosstab(
    df_test[col_label],
    df_test["pred_nb"],
    rownames=["Churn real"],
    colnames=["Churn previsto"]
)
print("\nMatriz de confusão (TESTE):")
print(confusion_test)

# Acurácia por classe
acc_por_classe = (
    df_test.assign(acerto=(df_test["pred_nb"] == df_test[col_label]).astype(int))
           .groupby(col_label)["acerto"]
           .mean()
)
print("\nAcurácia por classe (TESTE) (0 = não churn, 1 = churn):")
print(acc_por_classe)

In [0]:
# Quantos rótulos diferentes cada texto pode ter?
df = your_dataframe_hereambig = (

    df.groupby("CustomerFeedback_clean")["Churn"]
      .nunique()
      .value_counts()
)

print(ambig)

In [0]:
# =====================================================
# Naive Bayes Correto para Feedback de Clientes (Churn)
# - Usa treino/teste para evitar leakage
# - Calcula priors corretamente
# - Gera probabilidades, não só labels
# - Pode ser usado como meta-feature no modelo final
# =====================================================

import pandas as pd
import numpy as np
from collections import Counter
from math import log10
from sklearn.model_selection import train_test_split

# -----------------------------------------------------
# 1. Carregar dataset e preparar dados
# -----------------------------------------------------
df = pd.read_csv("history_clean.csv")

TEXT_COL = "CustomerFeedback_clean"
TARGET_COL = "Churn"

df = df.dropna(subset=[TEXT_COL])
df[TEXT_COL] = df[TEXT_COL].astype(str)
df[TARGET_COL] = df[TARGET_COL].astype(int)

# -----------------------------------------------------
# 2. Separar treino e teste (ESSENCIAL para evitar leakage!)
# -----------------------------------------------------
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df[TARGET_COL], 
    random_state=42
)

print("Treino:", len(train_df), " - Teste:", len(test_df))

# -----------------------------------------------------
# 3. Função que treina Naive Bayes com bag-of-words
# -----------------------------------------------------
def treinar_naive_bayes(df_train, text_col, label_col, alpha=1):

    pos_counter = Counter()
    neg_counter = Counter()

    total_pos_docs = 0
    total_neg_docs = 0

    # Tokenizar e contar palavras por classe
    for _, row in df_train.iterrows():
        label = row[label_col]
        tokens = row[text_col].lower().split()

        if label == 0:
            total_pos_docs += 1
            for w in tokens:
                pos_counter[w] += 1
        else:
            total_neg_docs += 1
            for w in tokens:
                neg_counter[w] += 1

    vocab = set(pos_counter.keys()) | set(neg_counter.keys())
    V = len(vocab)

    total_pos_words = sum(pos_counter.values())
    total_neg_words = sum(neg_counter.values())

    # Calcular priors corretos
    total_docs = total_pos_docs + total_neg_docs
    log_prior_pos = log10(total_pos_docs / total_docs)
    log_prior_neg = log10(total_neg_docs / total_docs)

    modelo = {
        "vocab": vocab,
        "pos_counter": pos_counter,
        "neg_counter": neg_counter,
        "total_pos_words": total_pos_words,
        "total_neg_words": total_neg_words,
        "log_prior_pos": log_prior_pos,
        "log_prior_neg": log_prior_neg,
        "V": V,
        "alpha": alpha
    }

    return modelo

# -----------------------------------------------------
# 4. Treinar o Naive Bayes no TREINO apenas
# -----------------------------------------------------
nb_model = treinar_naive_bayes(train_df, TEXT_COL, TARGET_COL)
print("Vocabulário:", nb_model["V"], "palavras")

# -----------------------------------------------------
# 5. Função de previsão (log-probabilidade + probabilidade)
# -----------------------------------------------------
def nb_pred_prob(texto, model):
    texto = str(texto).lower()
    tokens = texto.split()

    score_pos = model["log_prior_pos"]
    score_neg = model["log_prior_neg"]

    alpha = model["alpha"]
    V = model["V"]

    for w in tokens:
        # P(w|pos)
        c_pos = model["pos_counter"].get(w, 0)
        p_pos = (c_pos + alpha) / (model["total_pos_words"] + alpha * V)
        score_pos += log10(p_pos)

        # P(w|neg)
        c_neg = model["neg_counter"].get(w, 0)
        p_neg = (c_neg + alpha) / (model["total_neg_words"] + alpha * V)
        score_neg += log10(p_neg)

    # Converter scores logarítmicos em probabilidade real
    # usando logística (sigmoid)
    diff = score_neg - score_pos
    prob = 1 / (1 + 10**(-diff))

    return prob  # probabilidade de churn

def nb_pred_label(texto, model):
    prob = nb_pred_prob(texto, model)
    return 1 if prob >= 0.5 else 0

# -----------------------------------------------------
# 6. Validar no conjunto de teste (SEM LEAKAGE)
# -----------------------------------------------------
test_df["nb_prob"] = test_df[TEXT_COL].apply(lambda x: nb_pred_prob(x, nb_model))
test_df["nb_pred"] = (test_df["nb_prob"] >= 0.5).astype(int)

from sklearn.metrics import classification_report, roc_auc_score

print("\n=== DESEMPENHO DO NAIVE BAYES NO TESTE ===")
print(classification_report(test_df[TARGET_COL], test_df["nb_pred"]))
print("ROC-AUC:", roc_auc_score(test_df[TARGET_COL], test_df["nb_prob"]))

